In [ ]:
import cudf
import dask.dataframe as dd
import dask
dask.config.set({"dataframe.backend": "cudf"})


price_data = dd.read_csv('/home/edoardocame/Desktop/python_dir/eurgbp1m.csv')
price_data['sma'] = price_data['close'].rolling(60).mean()
price_data['upper_band'] = price_data['sma'] + price_data['close'].rolling(60).std() * 5
price_data['lower_band'] = price_data['sma'] - price_data['close'].rolling(60).std() * 5
price_data.tail()

In [ ]:
#PROTO-LOGIC BEHIND THE BOLLINGER BANDS PANDAS VERSION.



# Initialize positions
price_data['positions'] = 0


# Generate signals
price_data['long_entry'] = (price_data['close'] < price_data['lower_band'])
price_data['long_exit'] = (price_data['close'] > price_data['sma'])
price_data['short_entry'] = (price_data['close'] > price_data['upper_band'])
price_data['short_exit'] = (price_data['close'] < price_data['sma'])

def create_positions(df):
	df.loc[df['long_entry'], 'positions'] = 1
	df.loc[df['long_exit'], 'positions'] = 0
	df.loc[df['short_entry'], 'positions'] = -1
	df.loc[df['short_exit'], 'positions'] = 0
	return df

price_data = price_data.map_partitions(create_positions)

# Forward fill positions
price_data['positions'] = price_data['positions'].fillna(method='ffill').fillna(0)

# Record entry price
price_data['entry_price'] = price_data['close'].where(price_data['positions'].diff() != 0).fillna(method='ffill')

# Set stop loss percentage
stop_loss_percentage = 0.05

def apply_stop_loss(df, stop_loss_percentage):
	df.loc[(df['positions'] == 1) & (df['close'] < df['entry_price'] * (1 - stop_loss_percentage)), 'positions'] = 0
	df.loc[(df['positions'] == -1) & (df['close'] > df['entry_price'] * (1 + stop_loss_percentage)), 'positions'] = 0
	return df

# Apply stop loss using map_partitions
price_data = price_data.map_partitions(apply_stop_loss, stop_loss_percentage)

# Recalculate positions after stop loss
price_data['positions'] = price_data['positions'].fillna(method='ffill').fillna(0)

# Calculate returns
price_data['market_returns'] = price_data['close'].pct_change()
price_data['strategy_returns'] = price_data['market_returns'] * price_data['positions'].shift(1)